In [11]:
import sys
import argparse
import os
import pandas as pd
import datetime as dt
from datetime import timedelta
from datetime import datetime
import numpy as np
import json
import geopandas as gpd
from urllib.request import urlopen

In [12]:
world_cases = pd.read_csv("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv")
world_cases.head()

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,12/2/21,12/3/21,12/4/21,12/5/21,12/6/21,12/7/21,12/8/21,12/9/21,12/10/21,12/11/21
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,157387,157412,157431,157445,157499,157508,157542,157585,157858,157858
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,200639,201045,201402,201730,201902,202295,202641,202863,203215,203524
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,210921,211112,211297,211469,211662,211859,212047,212224,212434,212652
3,NaN,Andorra,42.50630,1.521800,0,0,0,0,0,0,...,17658,18010,18010,18010,18631,18815,18815,19272,19440,19440
4,NaN,Angola,-11.20270,17.873900,0,0,0,0,0,0,...,65208,65223,65244,65259,65259,65301,65332,65346,65371,65397


In [13]:
print(world_cases.columns)

Index(['Province/State', 'Country/Region', 'Lat', 'Long', '1/22/20', '1/23/20',
       '1/24/20', '1/25/20', '1/26/20', '1/27/20',
       ...
       '12/2/21', '12/3/21', '12/4/21', '12/5/21', '12/6/21', '12/7/21',
       '12/8/21', '12/9/21', '12/10/21', '12/11/21'],
      dtype='object', length=694)


In [14]:
# changing the columns to match the US aggregated data
world_cases = world_cases.drop(['Province/State', 'Lat', 'Long'], axis = 1)

# format the dates
dic = {}
for i in range(len(world_cases.columns)):
    to_replace = list(world_cases.columns)[i].replace('/', '-')
    year = to_replace[-2:]  
    year = "20" + year
    to_replace = to_replace[:len(to_replace) - 3]
    
    to_replace = year + '-' + to_replace
    
    dic[list(world_cases.columns)[i]] = to_replace
    

world_cases = world_cases.rename(index = str, columns = dic)
world_cases = world_cases.rename(columns = {"20on-Country-Reg": "name"})
world_cases.head()

,name,2020-1-22,2020-1-23,2020-1-24,2020-1-25,2020-1-26,2020-1-27,2020-1-28,2020-1-29,2020-1-30,...,2021-12-2,2021-12-3,2021-12-4,2021-12-5,2021-12-6,2021-12-7,2021-12-8,2021-12-9,2021-12-10,2021-12-11
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,157387,157412,157431,157445,157499,157508,157542,157585,157858,157858
1,Albania,0,0,0,0,0,0,0,0,0,...,200639,201045,201402,201730,201902,202295,202641,202863,203215,203524
2,Algeria,0,0,0,0,0,0,0,0,0,...,210921,211112,211297,211469,211662,211859,212047,212224,212434,212652
3,Andorra,0,0,0,0,0,0,0,0,0,...,17658,18010,18010,18010,18631,18815,18815,19272,19440,19440
4,Angola,0,0,0,0,0,0,0,0,0,...,65208,65223,65244,65259,65259,65301,65332,65346,65371,65397


In [15]:
# filter out the dates not needed
worldStart = dt.datetime(2020, 1, 22)
startDate = dt.datetime(2020, 2, 14)
endDate = dt.datetime(2021, 12, 9)
# print(int((endDate - startDate).days))

# datetime_columns = pd.to_datetime(world_cases.columns[1:])
# column_drop_list = [i for i in list(world_case.columns) if date]


In [16]:
len(world_cases)

280

In [17]:
# group by name
world_cases = pd.DataFrame(world_cases.groupby(by = ["name"], as_index = False).sum())
world_cases.head()

,name,2020-1-22,2020-1-23,2020-1-24,2020-1-25,2020-1-26,2020-1-27,2020-1-28,2020-1-29,2020-1-30,...,2021-12-2,2021-12-3,2021-12-4,2021-12-5,2021-12-6,2021-12-7,2021-12-8,2021-12-9,2021-12-10,2021-12-11
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,157387,157412,157431,157445,157499,157508,157542,157585,157858,157858
1,Albania,0,0,0,0,0,0,0,0,0,...,200639,201045,201402,201730,201902,202295,202641,202863,203215,203524
2,Algeria,0,0,0,0,0,0,0,0,0,...,210921,211112,211297,211469,211662,211859,212047,212224,212434,212652
3,Andorra,0,0,0,0,0,0,0,0,0,...,17658,18010,18010,18010,18631,18815,18815,19272,19440,19440
4,Angola,0,0,0,0,0,0,0,0,0,...,65208,65223,65244,65259,65259,65301,65332,65346,65371,65397


In [18]:
len(world_cases)

196

In [19]:
# convert cumulative to daily
for i in range(len(world_cases.columns) - 1, 1, -1):
    world_cases[world_cases.columns[i]] = world_cases[world_cases.columns[i]].sub(world_cases[world_cases.columns[i - 1]], axis = 0)
# world_cases.head()

In [20]:
world_cases = world_cases.drop(world_cases.columns[1:(1+int((startDate - worldStart).days))], axis = 1)
world_cases = world_cases.drop(world_cases.columns[(1+int((endDate - startDate).days)):], axis = 1)
world_cases.head()

,name,2020-2-14,2020-2-15,2020-2-16,2020-2-17,2020-2-18,2020-2-19,2020-2-20,2020-2-21,2020-2-22,...,2021-11-29,2021-11-30,2021-12-1,2021-12-2,2021-12-3,2021-12-4,2021-12-5,2021-12-6,2021-12-7,2021-12-8
0,Afghanistan,0,0,0,0,0,0,0,0,0,...,42,29,70,28,25,19,14,54,9,34
1,Albania,0,0,0,0,0,0,0,0,0,...,195,195,228,466,406,357,328,172,393,346
2,Algeria,0,0,0,0,0,0,0,0,0,...,192,187,192,198,191,185,172,193,197,188
3,Andorra,0,0,0,0,0,0,0,0,0,...,0,403,311,232,352,0,0,621,184,0
4,Angola,0,0,0,0,0,0,0,0,0,...,11,13,15,25,15,21,15,0,42,31


In [21]:
# merge with the data with geoid
world_geoid = pd.read_csv('data/georegion.csv')
print(world_geoid.columns)
world_cases = world_geoid.merge(world_cases, left_on = world_geoid.columns[0], right_on = "name", how = 'left')


# to_drop = []
# to_drop.append(len(world_cases.columns) - 1)
# for i in range(2, 25):
#     to_drop.append(i)
# world_cases = world_cases.drop(world_cases.columns[to_drop], axis = 1)

world_cases = world_cases.drop(world_cases.columns[2], axis = 1)
world_cases.head()

Index(['﻿geoName', 'geoRegion'], dtype='object')


,﻿geoName,geoRegion,2020-2-14,2020-2-15,2020-2-16,2020-2-17,2020-2-18,2020-2-19,2020-2-20,2020-2-21,...,2021-11-29,2021-11-30,2021-12-1,2021-12-2,2021-12-3,2021-12-4,2021-12-5,2021-12-6,2021-12-7,2021-12-8
0,Antigua and Barbuda,NORTH AMERICA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,1.0,0.0,1.0,0.0,3.0
1,Algeria,AFRICA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,192.0,187.0,192.0,198.0,191.0,185.0,172.0,193.0,197.0,188.0
2,Azerbaijan,ASIA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,857.0,1678.0,1795.0,1772.0,1703.0,1406.0,1394.0,585.0,1530.0,1210.0
3,Albania,EURPOE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,195.0,195.0,228.0,466.0,406.0,357.0,328.0,172.0,393.0,346.0
4,Armenia,ASIA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,189.0,398.0,502.0,558.0,399.0,419.0,327.0,95.0,240.0,410.0


In [22]:
len(world_cases)

238

In [23]:
# world_cases[world_cases['_merge'] != 'both']

In [24]:
len(world_cases)

238

In [25]:
output_cases = pd.read_csv("data/output_cases.csv")
output_cases.head()


# world_cases
# world_cases.to_csv("data/world_cases.csv", index = False)

,name,state,2020-02-14,2020-02-15,2020-02-16,2020-02-17,2020-02-18,2020-02-19,2020-02-20,2020-02-21,...,2021-11-29,2021-11-30,2021-12-01,2021-12-02,2021-12-03,2021-12-04,2021-12-05,2021-12-06,2021-12-07,2021-12-08
0,Phoenix-Mesa-Scottsdale,AZ,1,0,0,0,0,0,0,0,...,1305,2018,2231,2644,3502,4012,2646,2033,2218,2530
1,San Jose-Sunnyvale-Santa Clara,CA,2,0,0,0,0,0,0,0,...,784,192,193,306,333,0,0,899,203,176
2,San Francisco-Oakland-Hayward,CA,2,0,0,0,0,0,0,0,...,1422,256,243,704,570,0,0,1757,353,345
3,San Diego-Carlsbad,CA,1,0,0,0,0,0,0,0,...,391,1828,552,1049,654,0,0,1269,2410,721
4,Los Angeles-Long Beach-Anaheim,CA,2,0,0,0,0,0,0,0,...,2012,1082,1682,2254,2260,2290,1631,2542,1307,2032


In [26]:
# world_cases = pd.concat([output_cases, world_cases], axis = 0)
world_cases.columns


Index(['﻿geoName', 'geoRegion', '2020-2-14', '2020-2-15', '2020-2-16',
       '2020-2-17', '2020-2-18', '2020-2-19', '2020-2-20', '2020-2-21',
       ...
       '2021-11-29', '2021-11-30', '2021-12-1', '2021-12-2', '2021-12-3',
       '2021-12-4', '2021-12-5', '2021-12-6', '2021-12-7', '2021-12-8'],
      dtype='object', length=666)

In [27]:
output_cases.columns

Index(['name', 'state', '2020-02-14', '2020-02-15', '2020-02-16', '2020-02-17',
       '2020-02-18', '2020-02-19', '2020-02-20', '2020-02-21',
       ...
       '2021-11-29', '2021-11-30', '2021-12-01', '2021-12-02', '2021-12-03',
       '2021-12-04', '2021-12-05', '2021-12-06', '2021-12-07', '2021-12-08'],
      dtype='object', length=666)

In [28]:
rename_col = {}
for i in range(len(output_cases.columns)):
    rename_col[list(output_cases.columns)[i]] = list(world_cases.columns)[i]
output_cases = output_cases.rename(index = str, columns = rename_col)
# print(rename_col)
# print(output_cases.columns == world_cases.columns)

In [29]:
world_cases = world_cases.append(output_cases, ignore_index = True)

In [30]:
world_cases.head()

,﻿geoName,geoRegion,2020-2-14,2020-2-15,2020-2-16,2020-2-17,2020-2-18,2020-2-19,2020-2-20,2020-2-21,...,2021-11-29,2021-11-30,2021-12-1,2021-12-2,2021-12-3,2021-12-4,2021-12-5,2021-12-6,2021-12-7,2021-12-8
0,Antigua and Barbuda,NORTH AMERICA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,5.0,1.0,0.0,1.0,0.0,3.0
1,Algeria,AFRICA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,192.0,187.0,192.0,198.0,191.0,185.0,172.0,193.0,197.0,188.0
2,Azerbaijan,ASIA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,857.0,1678.0,1795.0,1772.0,1703.0,1406.0,1394.0,585.0,1530.0,1210.0
3,Albania,EURPOE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,195.0,195.0,228.0,466.0,406.0,357.0,328.0,172.0,393.0,346.0
4,Armenia,ASIA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,189.0,398.0,502.0,558.0,399.0,419.0,327.0,95.0,240.0,410.0


In [31]:
georegion_af = pd.read_csv("./data/georegion_af.csv")

In [32]:
georegion_af = georegion_af[georegion_af['GEOID'] != -9999]

In [33]:
georegion_af.head()

,GEOID,geoName,geoRegion,geoLON,geoLAT,geoISO3
1,1,Antigua and Barbuda,NORTH AMERICA,-61.791,17.280,ATG
2,2,Algeria,AFRICA,2.632,28.163,DZA
3,3,Azerbaijan,ASIA,47.532,40.292,AZE
4,4,Albania,EURPOE,20.068,41.143,ALB
5,5,Armenia,ASIA,44.948,40.286,ARM


In [34]:
len(georegion_af)

1168

In [35]:
world_cases.columns

Index(['﻿geoName', 'geoRegion', '2020-2-14', '2020-2-15', '2020-2-16',
       '2020-2-17', '2020-2-18', '2020-2-19', '2020-2-20', '2020-2-21',
       ...
       '2021-11-29', '2021-11-30', '2021-12-1', '2021-12-2', '2021-12-3',
       '2021-12-4', '2021-12-5', '2021-12-6', '2021-12-7', '2021-12-8'],
      dtype='object', length=666)

In [36]:
world_cases = world_cases.merge(georegion_af, left_on = [world_cases.columns[0], "geoRegion"] , right_on = ["geoName", "geoRegion"], how = 'right')

In [37]:
# reorder the columns
columns_reordered1 = list(world_cases.columns)[1:2]
columns_reordered2 = list(world_cases.columns)[2:-6]
columns_reordered3 = list(world_cases.columns)[-5:-3]
columns_reordered4 = list(world_cases.columns)[-3:]
columns_reordered = columns_reordered3 + columns_reordered1 + columns_reordered4 + columns_reordered2 


world_cases = world_cases.reindex(columns = columns_reordered)

In [38]:
# change the floats to integers
def float_to_int(x):
    if (x != x):
         # print("hey")
        return -9999
    x = int(x)
    return x

for i in list(world_cases.columns[6:]):
    world_cases[i] = world_cases[i].apply(float_to_int)

def format_float(x):
    x = round(x, 3)
    return x

world_cases['geoLON'] = world_cases['geoLON'].apply(format_float)
world_cases['geoLAT'] = world_cases['geoLAT'].apply(format_float)
    

In [39]:
world_cases.head()

,GEOID,geoName,geoRegion,geoLON,geoLAT,geoISO3,2020-2-14,2020-2-15,2020-2-16,2020-2-17,...,2021-11-28,2021-11-29,2021-11-30,2021-12-1,2021-12-2,2021-12-3,2021-12-4,2021-12-5,2021-12-6,2021-12-7
0,1,Antigua and Barbuda,NORTH AMERICA,-61.791,17.280,ATG,0,0,0,0,...,0,0,0,0,0,5,1,0,1,0
1,2,Algeria,AFRICA,2.632,28.163,DZA,0,0,0,0,...,172,192,187,192,198,191,185,172,193,197
2,3,Azerbaijan,ASIA,47.532,40.292,AZE,0,0,0,0,...,1662,857,1678,1795,1772,1703,1406,1394,585,1530
3,4,Albania,EURPOE,20.068,41.143,ALB,0,0,0,0,...,418,195,195,228,466,406,357,328,172,393
4,5,Armenia,ASIA,44.948,40.286,ARM,0,0,0,0,...,409,189,398,502,558,399,419,327,95,240


In [40]:
len(world_cases)

1168

In [41]:
world_cases.to_csv("data/world_cases.csv", index = False)